# Il giro del mondo in 80 giorni

Questo progetto utilizza un dataset contenente le principali città del mondo, con annessa locazione geografica e altre informazioni, per calcolare il tempo minimo che ci si metterebbe a viaggiare tra due città, il percorso migliore, il percorso più turistico e altre funzionalità.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from geopy import distance as geopy
import csv
import time

In [2]:
data_file = './data/worldcities_ascii.csv'
with open(data_file, 'r') as f:
    reader = csv.reader(f)
    all_lines = []
    for row in reader:
        all_lines.append(row)

## Preprocessing

Preprocesso i dati pulendo le virgole in eccesso, corregendo qualche nome e aggiungendo i campi che mi servono.

>TODO: Aggiungere dei campi tipo cose da visitare, prezzo medio ecc...

In [3]:
newLines = []
for line in all_lines:
    newLine = []
    for field in line:
        field = field.replace("Korea, South", "South Korea")
        field = field.replace("Korea, North", "North Korea")
        field = field.replace("Gambia, The", "The Gambia")
        field = field.replace("Micronesia, Federated States Of", "Federated States Of Micronesia")
        field = field.replace("Bahamas, The", "The Bahamas")
        field = field.replace("Saint Helena, Ascension, And Tristan Da Cunha", "Saint Helena, Ascension and Tristan da Cunha")
        field = field.replace("Islamorada, Village of Islands", "Village of Islands Islamorada")
        field = field.replace("\n", "")
        newLine.append(field)
    newLines.append(newLine)

#Lo esporto in caso mi servisse in altri progetti
pd.DataFrame(newLines[1:], columns=newLines[0]).to_csv("./data/worldcities_preprocessed.csv", index=False)

In [4]:
#Siccome sono troppe per lavorarci inizio a prenderne solo 2000 a caso
lineIndexes = np.random.choice(range(1, len(newLines)), size=2000, replace=False)
df = pd.DataFrame([newLines[i] for i in lineIndexes], columns=newLines[0])
df

,city,lat,lng,country,iso3,population,id
0,Dosso,13.05,3.2,Niger,NER,49750,1562574611
1,Dayton,35.4912,-85.012,United States,USA,10410,1840014525
2,Busesa,0.6263,33.6003,Uganda,UGA,,1800175659
3,Rapid City,44.0716,-103.2204,United States,USA,92664,1840002180
4,Hermosa Beach,33.8654,-118.3966,United States,USA,19320,1840020487
...,...,...,...,...,...,...,...
1995,Malpils,57.0101,24.9578,Latvia,LVA,,1428737728
1996,Lang Suan,9.9519,99.0813,Thailand,THA,11250,1764676997
1997,Nova Friburgo,-22.26,-42.54,Brazil,BRA,171991,1076922983
1998,Baicheng,23.901,106.6194,China,CHN,3466758,1156981113


In [5]:
class CityNode:
    
    def __init__(self, cityID: int, cityName: str, lat: float, lng: float, population: int, countryISO3: str):
        self.cityID = cityID
        self.cityName = cityName
        self.lat = lat
        self.lng = lng
        self.coordinates = np.array([lat, lng])
        self.population = population
        self.countryISO3 = countryISO3
        self.neighbour1 = None
        self.neighbour2 = None
        self.neighbour3 = None
    
    def updateNeighbour(self, city, position: int):
        if position == 1:
            self.neighbour1 = city
        elif position == 2:
            self.neighbour2 = city
        elif position == 3:
            self.neighbour3 = city
    
    def isEligible(self, city):
        if self.neighbour3 is None:
            return true
        
            
    def __repr__(self):
        return f"{self.cityName} at {self.lat}, {self.lng}"

In [6]:
graph = [CityNode(int(c.id), c.city, float(c.lat), float(c.lng), c.population, c.iso3) for _, c in df.iterrows()]

In [7]:
def updateMin(min1, min2, min3, dist: float, index: int):
    if (min1 is None) or (min2 is None) or (min3 is None):
        min1 = {"index": index, "dist": dist}
        min2 = {"index": index, "dist": dist}
        min3 = {"index": index, "dist": dist}
    elif dist <= min1["dist"]:
        min3 = min2
        min2 = min1
        min1 = {"index": index, "dist": dist}
    elif dist <= min2["dist"]:
        min3 = min2
        min2 = {"index": index, "dist": dist}
    elif dist <= min3["dist"]:
        min3 = {"index": index, "dist": dist}
    return (min1, min2, min3)

In [8]:
start = time.time()
for i in range(len(graph)):
    min1 = None
    min2 = None
    min3 = None
    for j in range(len(graph)):
        if i == j:
            continue
        dist = ((graph[i].lat-graph[j].lat)**2+(graph[i].lng-graph[j].lng)**2)**(1/2)
        #dist = geopy.distance(graph[i].coordinates, graph[j].coordinates).km
        min1, min2, min3 = updateMin(min1, min2, min3, dist, j)
    graph[i].updateNeighbour(graph[min1["index"]], 1)
    graph[i].updateNeighbour(graph[min2["index"]], 2)
    graph[i].updateNeighbour(graph[min3["index"]], 3)
    if i % 1000 == 0:
        print(f"Execution: {i}\t\tTime elapsed (s): {time.time() - start}")

Execution: 0		Time elapsed (s): 0.0029993057250976562
Execution: 1000		Time elapsed (s): 2.751925230026245


## Come procedere

1. Decidere grandezza quadrati per dividere mondo
2. Assegnare ogni città a quadrato
3. Per ogni città in ogni quadrato trovare 3 città più vicine cercando nello stesso quadrato della città e nei 9 intorno
4. In caso non ci siano almeno 4 città in questi 10 quadrati cercare anche nei 16 quadrati più esterni
5. Scegliere rappresentazione migliore grafo e calcolare pesi degli archi
6. Scegliere algoritmo di navigazione grafo
7. Implementare metodi per fare query di vario tipo

In [18]:
groups = np.zeros((180, 360))
for city in graph:
    i = int((city.lat+90)/10)
    j = int((city.lng+180)/10)
    print(i, j)
    groups[i][j] = city

10 18


TypeError: float() argument must be a string or a number, not 'CityNode'